In [1]:
import numpy as np
import sys
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataset import dataset_MOT_segmented
from models import vqvae
from options import option_vq

In [2]:
# Create the data loader (adjust window_size, batch_size as needed)
loader = dataset_MOT_segmented.addb_data_loader(data_dir='/home/mnt/AddBiomechanics', mode='train')

  0%|          | 0/56 [00:00<?, ?it/s]

100%|██████████| 56/56 [03:29<00:00,  3.74s/it]

Total number of motions: 851


In [3]:
subject_summary = {}

for batch in loader:
    if isinstance(batch, (list, tuple)) and len(batch) == 4:
        subject_names = batch[3]
        names = batch[2]
        if isinstance(subject_names, (list, tuple)):
            for subj, name in zip(subject_names, names):
                if "::trial" not in name:
                    print(f"subject: {subj}\n  name: {name}")
                    continue

                b3d_path, trial_str = name.split("::trial")
                trial_idx = int(trial_str)
                try:
                    import nimblephysics as nimble
                    subj_on_disk = nimble.biomechanics.SubjectOnDisk(b3d_path)
                    trial_name = subj_on_disk.getTrialName(trial_idx)
                    trial_orig = subj_on_disk.getTrialOriginalName(trial_idx)

                    # Parse activity differently based on paper
                    if subj.startswith("Camargo2021"):
                        parts = trial_name.split("_")
                        if len(parts) > 0:
                            if parts[0] == "levelground":
                                activity = "levelground"
                            elif parts[0] == "ramp":
                                activity = "ramp"
                            elif parts[0] == "stair":
                                activity = "stair"
                            elif parts[0] == "treadmill":
                                activity = "treadmill"
                            else:
                                activity = parts[0]
                        else:
                            activity = "unknown"
                        trial_key = trial_orig
                    elif subj.startswith("Carter2023"):
                        parts = trial_orig.split("_")
                        if parts[0] in ["walk", "flatrun", "uphillrun"]:
                            activity = parts[0]
                        else:
                            activity = parts[0]
                        trial_key = trial_orig
                    elif subj.startswith("Falisse2017"):
                        parts = trial_name.split("_")
                        activity = parts[0] if len(parts) > 0 else "unknown"
                        trial_key = trial_orig 
                    elif subj.startswith("Fregly2012"):
                        parts = trial_name.split("_")
                        if len(parts) > 1:
                            if parts[0] == "SC":
                                activity = re.sub(r'\d+$', '', parts[1].lower())
                            else:
                                activity = re.sub(r'\d+$', '', parts[0].lower())
                        else:
                            activity = "unknown"
                        trial_key = trial_orig
                    elif subj.startswith("Hamner2013"):
                        parts = trial_name.split("_")
                        activity = re.sub(r'\d+', '', parts[0]) if len(parts) > 0 else "unknown"
                        trial_key = trial_orig
                    elif subj.startswith("Han2023"): 
                        parts = trial_name.split("_")
                        activity = re.sub(r'\d+', '', parts[1]) if len(parts) > 0 else "unknown"
                        trial_key = trial_orig 
                    elif subj.startswith("Li2021"):
                        parts = trial_name.split("_")
                        activity = "walking"
                        trial_key = trial_orig
                    elif subj.startswith("Moore2015"):
                        parts = trial_name.split("_")
                        activity = "walking"
                        trial_key = trial_orig
                    elif subj.startswith("Santos2017"):
                        parts = trial_name.split("_")
                        activity = "standing"
                        trial_key = trial_orig
                    elif subj.startswith("Tan2021"):
                        parts = trial_orig.split("_")
                        if len(parts) >= 2:
                            activity = f"{parts[0]}_{parts[1]}"
                        else:
                            activity = parts[0] if len(parts) > 0 else "unknown"
                        trial_key = trial_orig
                    elif subj.startswith("Tan2022"):
                        parts = trial_orig.split("_")
                        if len(parts) >= 2:
                            activity = f"{parts[0]}_{parts[1]}"
                        else:
                            activity = parts[0] if len(parts) > 0 else "unknown"
                        trial_key = trial_orig
                    elif subj.startswith("Tiziana2019"):
                        parts = trial_name.split("_")
                        activity = re.sub(r'\d+', '', parts[0]) if len(parts) > 0 else "unknown"
                        trial_key = trial_orig
                    elif subj.startswith("Uhlrich2023"):
                        parts = trial_name.split("_")
                        activity = re.sub(r'\d+', '', parts[0]) if len(parts) > 0 else "unknown"
                        trial_key = trial_orig
                    elif subj.startswith("vanderZee2022"):
                        parts = trial_name.split("_")
                        activity = "walking"
                        trial_key = trial_orig
                    elif subj.startswith("Wang2023"):
                        parts = trial_orig.split("_")
                        activity = parts[1] if len(parts) > 1 else "unknown"
                        trial_key = trial_orig 
                    else:
                        # Default: first part
                        parts = trial_name.split("_")
                        activity = parts[0] if len(parts) > 0 else "unknown"
                        trial_key = trial_orig

                    if subj not in subject_summary:
                        subject_summary[subj] = {"b3d": b3d_path, "activities": {}}
                    # Use a dict to ensure unique trials per activity (by original name)
                    if activity not in subject_summary[subj]["activities"]:
                        subject_summary[subj]["activities"][activity] = {}
                    # Only add if not already present (merges segments for Wang2023)
                    if trial_key not in subject_summary[subj]["activities"][activity]:
                        subject_summary[subj]["activities"][activity][trial_key] = trial_orig
                except Exception as e:
                    print(f"subject: {subj}\n  b3d: {b3d_path}\n  trial: {trial_idx}\n  error: {e}")
        else:
            print("subject_name:", subject_names)
    else:
        print("No subject_info found in batch.")

from collections import defaultdict


paper_summary = defaultdict(lambda: {"activities": set(), "trials": 0, "activity_counts": defaultdict(int)})

for subj, info in subject_summary.items():
    # Extract paper prefix (e.g., Han2023, Falisse2017, etc.)
    if "/" in subj:
        paper = subj.split("/")[0]
    elif "_" in subj:
        paper = subj.split("_")[0]
    else:
        paper = subj
    for activity, trials in info["activities"].items():
        paper_summary[paper]["activities"].add(activity)
        paper_summary[paper]["activity_counts"][activity] += len(trials)
        paper_summary[paper]["trials"] += len(trials)

report_lines = []

# --- Print paper-level summary only ---
for paper, summary in sorted(paper_summary.items()):
    report_lines.append(f"{paper}:")
    report_lines.append(f"Activities: {len(summary['activities'])}")
    report_lines.append(f"Trials: {summary['trials']}")
    for activity, count in sorted(summary["activity_counts"].items()):
        report_lines.append(f"  {activity}: {count}")
    report_lines.append("")

    # Also print to stdout
    print(f"{paper}:")
    print(f"Activities: {len(summary['activities'])}")
    print(f"Trials: {summary['trials']}")
    for activity, count in sorted(summary["activity_counts"].items()):
        print(f"  {activity}: {count}")
    print()

# --- Existing subject-level printout ---
for subj, info in sorted(subject_summary.items()):
    report_lines.append(f"Subject: {subj}")
    report_lines.append(f"File Paths: {info['b3d']}")
    report_lines.append(f"Activities: {len(info['activities'])}")
    report_lines.append(f"Trials: {sum(len(trials) for trials in info['activities'].values())}")
    for activity, trials in sorted(info["activities"].items()):
        report_lines.append(f"  {activity}: {len(trials)}")
    report_lines.append("")

    # Also print to stdout
    print(f"Subject: {subj}")
    print(f"File Paths: {info['b3d']}")
    print(f"Activities: {len(info['activities'])}")
    print(f"Trials: {sum(len(trials) for trials in info['activities'].values())}")
    for activity, trials in sorted(info["activities"].items()):
        print(f"  {activity}: {len(trials)}")
    print()

# --- Save to report.txt ---
with open("report.txt", "w") as f:
    for line in report_lines:
        f.write(line + "\n")

Wang2023:
Activities: 7
Trials: 120
  jump: 9
  land: 9
  lunge: 9
  run: 27
  squat: 9
  static: 3
  walk: 54

Subject: Wang2023/Subj04
File Paths: /home/mnt/AddBiomechanics/train/No_Arm/Wang2023_Formatted_No_Arm/Subj04/Subj04.b3d
Activities: 6
Trials: 13
  jump: 1
  land: 1
  lunge: 1
  run: 3
  squat: 1
  walk: 6

Subject: Wang2023/Subj05
File Paths: /home/mnt/AddBiomechanics/train/No_Arm/Wang2023_Formatted_No_Arm/Subj05/Subj05.b3d
Activities: 6
Trials: 13
  jump: 1
  land: 1
  lunge: 1
  run: 3
  squat: 1
  walk: 6

Subject: Wang2023/Subj06
File Paths: /home/mnt/AddBiomechanics/train/No_Arm/Wang2023_Formatted_No_Arm/Subj06/Subj06.b3d
Activities: 6
Trials: 13
  jump: 1
  land: 1
  lunge: 1
  run: 3
  squat: 1
  walk: 6

Subject: Wang2023/Subj07
File Paths: /home/mnt/AddBiomechanics/train/No_Arm/Wang2023_Formatted_No_Arm/Subj07/Subj07.b3d
Activities: 6
Trials: 13
  jump: 1
  land: 1
  lunge: 1
  run: 3
  squat: 1
  walk: 6

Subject: Wang2023/Subj08
File Paths: /home/mnt/AddBiomechani

In [ ]:
# # Print all trials and their names for Uhlrich2023
# subject_summary = {}

# for batch in loader:
#     if isinstance(batch, (list, tuple)) and len(batch) == 4:
#         subject_names = batch[3]
#         names = batch[2]
#         if isinstance(subject_names, (list, tuple)):
#             for subj, name in zip(subject_names, names):
#                 # if not subj.startswith("Han2023"):
#                 #     continue
#                 if "::trial" in name:
#                     b3d_path, trial_str = name.split("::trial")
#                     trial_idx = int(trial_str)
#                     try:
#                         import nimblephysics as nimble
#                         subj_on_disk = nimble.biomechanics.SubjectOnDisk(b3d_path)
#                         trial_name = subj_on_disk.getTrialName(trial_idx)
#                         trial_orig = subj_on_disk.getTrialOriginalName(trial_idx)
#                         if subj not in subject_summary:
#                             subject_summary[subj] = {"b3d": b3d_path, "trials": []}
#                         subject_summary[subj]["trials"].append((trial_name, trial_orig))
#                     except Exception as e:
#                         print(f"subject: {subj}\n  b3d: {b3d_path}\n  trial: {trial_idx}\n  error: {e}")
#                 else:
#                     print(f"subject: {subj}\n  name: {name}")
#         else:
#             print("subject_name:", subject_names)
#     else:
#         print("No subject_info found in batch.")

# for subj, info in sorted(subject_summary.items()):
#     print(f"Subject: {subj}")
#     print(f"File Path: {info['b3d']}")
#     print(f"Trials: {len(info['trials'])}")
#     for trial_name, trial_orig in sorted(info["trials"]):
#         print(f"  {trial_name} (original: {trial_orig})")
#     print()

Subject: vanderZee2022/p3
File Path: /home/mnt/AddBiomechanics/test/No_Arm/vanderZee2022_Formatted_No_Arm/p3/p3.b3d
Trials: 131
  trial10_segment_0 (original: trial10)
  trial10_segment_1 (original: trial10)
  trial10_segment_2 (original: trial10)
  trial10_segment_3 (original: trial10)
  trial11_segment_0 (original: trial11)
  trial11_segment_1 (original: trial11)
  trial11_segment_2 (original: trial11)
  trial11_segment_3 (original: trial11)
  trial12_segment_0 (original: trial12)
  trial12_segment_1 (original: trial12)
  trial12_segment_2 (original: trial12)
  trial12_segment_3 (original: trial12)
  trial13_segment_0 (original: trial13)
  trial13_segment_1 (original: trial13)
  trial13_segment_2 (original: trial13)
  trial13_segment_3 (original: trial13)
  trial14_segment_0 (original: trial14)
  trial14_segment_1 (original: trial14)
  trial14_segment_2 (original: trial14)
  trial14_segment_3 (original: trial14)
  trial15_segment_0 (original: trial15)
  trial15_segment_1 (original: t